# Using FunctionTools with an Agent

<h3> Here we will see how we can use Functions which also links to real world datas tools </h3>
<h3> By default Agent only knows about the data it has trained on, it does not know the real world data or the propietary data</h3>
<h3> Here we can leverage functions as tools for the agent and execute the code </h3>

In [69]:
# Installation refer 'basic_food_agent.ipynb' , installation section
# !pip install -U agent-framework --pre
# You can see the basic_food_agent.ipynb for basic agent run

<h4> Here we will use meals db API from where we can get the random meal and data</h4>

In [70]:
import requests
import json
from typing import Annotated, List, Dict, Any, Optional
from pydantic import Field

# Clean the MealDB response for the LLM
def _clean_meal_data(meal: Dict[str, Any]) -> Dict[str, Any]:
    """
    Helper function to restructure the raw meal API response into a clean, 
    LLM-friendly format by combining ingredients and measures.
    """
    if not meal:
        return {}

    # Combine ingredients and measures into a single list
    ingredients = []
    for i in range(1, 21):
        ing = meal.get(f"strIngredient{i}")
        measure = meal.get(f"strMeasure{i}")
        if ing and ing.strip():
            ingredients.append(f"{measure.strip()} {ing.strip()}".strip())

    return {
        "id": meal.get("idMeal"),
        "name": meal.get("strMeal"),
        "category": meal.get("strCategory"),
        "area": meal.get("strArea"),
        "instructions": meal.get("strInstructions"),
        "ingredients": ingredients,
        "tags": meal.get("strTags"),
        "youtube_link": meal.get("strYoutube")
    }

# Get random meal for today
def get_random_meal() -> str:
    """
    Retrieves a random meal recipe from the database. 
    Useful when the user wants a surprise suggestion or explicitly asks for a random recommendation.

    Returns:
        str: A JSON string containing the meal name, ingredients, and cooking instructions.
    """
    try:
        response = requests.get("https://www.themealdb.com/api/json/v1/1/random.php")
        response.raise_for_status()
        data = response.json()
        
        if not data.get("meals"):
            return json.dumps({"error": "No meal found."})
            
        meal = _clean_meal_data(data["meals"][0])
        return json.dumps(meal, indent=2)
        
    except Exception as e:
        return json.dumps({"error": f"Failed to fetch random meal: {str(e)}"})

# Here we get the meal by name
def get_meal_by_name(
    meal_name: Annotated[str, Field(description="The name of the meal to search for (e.g., 'Arrabiata', 'Burger').")]
) -> str:
    """
    Searches for a specific meal recipe by name. 
    Use this when the user asks for a specific dish or wants to know how to cook a named item.

    Args:
        meal_name: The name of the dish to search for.

    Returns:
        str: A JSON string containing a list of matching meals with their details.
    """
    try:
        # The API requires a search query parameter 's'
        response = requests.get(f"https://www.themealdb.com/api/json/v1/1/search.php?s={meal_name}")
        response.raise_for_status()
        data = response.json()
        
        if not data.get("meals"):
            return json.dumps({"status": "not_found", "message": f"No meals found with the name '{meal_name}'."})
        
        # Clean and limit results (e.g., top 3 matches to save tokens)
        results = [_clean_meal_data(m) for m in data["meals"][:3]]
        return json.dumps(results, indent=2)

    except Exception as e:
        return json.dumps({"error": f"Failed to search for meal: {str(e)}"})

In [71]:
# Import core dependencies to create the agent, for Agent Framework
import asyncio
import os
import json

from dotenv import load_dotenv, find_dotenv
# Core components for building Agent, tool-enabled agents
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient

In [72]:
# load environment file
load_dotenv(find_dotenv())

True

In [73]:
# Setup OpenAIChatClient for LLM Inference - Here we will use OpenRouter API which is compatible with OpenAI and NVIDIA 30B model
# This client connects to the OpenRouter Models which are OpenAI-compatible endpoint
# Environment variables required
# OPENROUTER_ENDPOINT - 
# OPENROUTER_API_KEY
openai_chat_client = OpenAIChatClient(
    base_url=os.environ.get("OPENROUTER_ENDPOINT"),
    api_key=os.environ.get("OPENROUTER_API_KEY"),
    model_id="nvidia/nemotron-3-nano-30b-a3b:free"
)

<h2> Here we configure the System Prompt for Agent with Name</h2>

In [74]:
AGENT_NAME = "FoodAgent"

AGENT_INSTRUCTIONS = """You are an expert AI Chef dedicated to helping users discover and prepare delicious meals.

CORE BEHAVIORS:
1. **Tool Usage**: You have access to a recipe database. ALWAYS use the provided tools to answer questions about recipes. Do not guess or hallucinate ingredients.
   - Use `get_meal_by_name` when the user asks for a specific dish.
   - Use `get_random_meal` when the user is undecided, asks for a suggestion, or wants a surprise.

2. **Response Format**: 
   - Start with an appetizing description of the dish.
   - List key ingredients clearly (based on the tool output).
   - Summarize the cooking instructions to be easy to follow.
   - If the tool provides a YouTube link, always include it at the end.
   - Include Tool Name used for fetching response

3. **Constraints**: 
   - Keep your response friendly but strictly under 200 words. 
   - If instructions are long, summarize the key steps to fit the word limit.
"""

In [75]:
# create the agent remember we are not using any tools here, this is simple example
agent = ChatAgent(
    name = AGENT_NAME,
    chat_client=openai_chat_client,
    instructions=AGENT_INSTRUCTIONS,
    tools=[get_random_meal, get_meal_by_name]
)

In [76]:
response1 = await agent.run("Help me something to eat today?")

<h3> You can clearly see the Agent ouputs the Tool used </h3>

In [77]:
print(response1.text)

Looking for a hearty, flavor‑packed dish? **Bigos (Polish hunter’s stew)** is a slow‑cooked beef stew brimming with sauerkraut, smoked meats, and aromatic spices—perfect for a comforting meal that tastes even better the next day.

**Key ingredients**
- White cabbage & beef stock  
- Mixed mushrooms  
- German sausages & bacon  
- Beef chunks, prunes, onion  
- Red wine, tomato purée, bay leaf, cloves, peppercorns, juniper & allspice berries  

**Quick steps**  
1️⃣ Simmer sliced cabbage with stock until tender (≈50 min).  
2️⃣ In a separate pan, fry sausage, bacon, and onion; add mushrooms, meat, prunes, and their soaking liquid.  
3️⃣ Return everything to the pot, stir in spices, wine, and tomato purée, then simmer covered for ~1 hour. Cool, store, and reheat thoroughly before serving.  

Fetched via **get_random_meal**.  
Watch the cooking video here: https://www.youtube.com/watch?v=Oqg_cO4s8ik


<h3> But we want to verify indeed the Agent is using the Function Tool or it's fabricating the response </h3>
<h3> Let's add middleware to verify this</h3>

In [78]:
# Here we add the logging Function middleware since we are working with functions
from agent_framework import AgentRunContext, FunctionInvocationContext
from typing import Callable , Awaitable

# Function middleware 
async def logging_function_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """Middleware that logs function calls."""
    print(f"Calling function: {context.function.name}")

    await next(context)

    print(f"Function result: {context.result}")


In [79]:
agent2 = ChatAgent(
    name = AGENT_NAME,
    chat_client=openai_chat_client,
    instructions=AGENT_INSTRUCTIONS,
    tools=[get_random_meal, get_meal_by_name],
    middleware=[logging_function_middleware]
)

<h3> You can clearly see the logging statement `Calling Function` which calls the `get_meal_by_name` function </h3>

In [80]:
result = await agent2.run("How to make pasta!")

Calling function: get_meal_by_name
Function result: [
  {
    "id": "52777",
    "name": "Mediterranean Pasta Salad",
    "category": "Seafood",
    "area": "Italian",
    "instructions": "Bring a large saucepan of salted water to the boil\r\nAdd the pasta, stir once and cook for about 10 minutes or as directed on the packet.\r\nMeanwhile, wash the tomatoes and cut into quarters. Slice the olives. Wash the basil.\r\nPut the tomatoes into a salad bowl and tear the basil leaves over them. Add a tablespoon of olive oil and mix.\r\nWhen the pasta is ready, drain into a colander and run cold water over it to cool it quickly.\r\nToss the pasta into the salad bowl with the tomatoes and basil.\r\nAdd the sliced olives, drained mozzarella balls, and chunks of tuna. Mix well and let the salad rest for at least half an hour to allow the flavours to mingle.\r\nSprinkle the pasta with a generous grind of black pepper and drizzle with the remaining olive oil just before serving.",
    "ingredients":

In [81]:
print(result.text)

Imagine a bright, Mediterranean‑style pasta salad bursting with juicy plum tomatoes, fragrant basil, briny olives, creamy mozzarella, and savory tuna—tossed with buttery farfalle.  

**Key ingredients:** 350 g farfalle, 200 g mozzarella balls, 250 g baby plum tomatoes, 1 bunch fresh basil, 40 g green olives, 200 g tuna, 3 Tbsp extra‑virgin olive oil, salt and pepper.  

**Quick steps:**  
1️⃣ Cook the pasta in salted water until al dente (≈10 min); drain and rinse under cold water.  
2️⃣ Halve the tomatoes, slice olives, tear basil, and whisk with olive oil.  
3️⃣ In a large bowl combine pasta, tomatoes, basil, olives, mozzarella, and tuna.  
4️⃣ Mix well, let rest 30 min for flavors to meld, then finish with a grind of pepper and a drizzle of oil.  

Enjoy this refreshing dish cold or at room temperature.  

Watch the full preparation here: https://www.youtube.com/watch?v=e52IL8zYmaE  

*(Recipe fetched with **get_meal_by_name**.)*


<h3> You can clearly see the logging statement `Calling Function` which calls the `get_random_meal` function </h3>

In [82]:
result2 = await agent2.run("What should I eat today?")

Calling function: get_random_meal
Function result: {
  "id": "52769",
  "name": "Kapsalon",
  "category": "Lamb",
  "area": "Dutch",
  "instructions": "Cut the meat into strips. Heat oil in a pan and fry the strips for 6 minutes until it's ready.\r\nBake the fries until golden brown in a deep fryrer. When ready transfer to a backing dish. Make sure the fries are spread over the whole dish.\r\nCover the fries with a new layer of meat and spread evenly.\r\nAdd a layer of cheese over the meat. You can also use grated cheese. When done put in the oven for a few minutes until the cheese is melted.\r\nChop the lettuce, tomato and cucumber in small pieces and mix together. for a basic salad. As extra you can add olives jalapenos and a red union.\r\nDived the salad over the dish and Serve with garlicsauce and hot sauce",
  "ingredients": [
    "250 Grams Fries",
    "500 Grams Doner Meat",
    "Topping Garlic sauce",
    "Topping Hotsauce",
    "1 Bulb Lettuce",
    "1 Tomato",
    "3rd Cucumb

In [83]:
print(result2.text)

**Meal:** Kapsalon – a Dutch‑style lamb doner bake with crispy fries, melty Gouda, and a fresh veggie salad.

**Key Ingredients**  
- 250 g frozen fries  
- 500 g doner (lamb) strips  
- 100 g Gouda cheese, grated  
- Lettuce, tomato, cucumber (for the salad)  
- Garlic sauce & hot sauce (for serving)

**Quick Directions**  
1. Fry the lamb strips in oil for ~6 min until browned.  
2. Deep‑fry the fries until golden, then spread them in a baking dish.  
3. Layer the cooked meat over the fries, sprinkle the cheese on top, and bake a few minutes until the cheese melts.  
4. Toss chopped lettuce, tomato, cucumber (and optional olives, jalapeños, red onion) with a little dressing.  
5. Plate the salad over the cheesy meat‑and‑fries, drizzle with garlic sauce and hot sauce, and enjoy!

**Video guide:** https://www.youtube.com/watch?v=UIcuiU1kV8I  

*Found using the “get_random_meal” tool.*
